# Leitura de nota de corretagem

É necessário que o arquivo seja convertido em xml

In [52]:
pip install correpy

Note: you may need to restart the kernel to use updated packages.


In [106]:
import io

from correpy.parsers.brokerage_notes.b3_parser.b3_parser import B3Parser

import numpy as np
import pandas as pd
import seaborn as sns


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Carregamento de dados

In [54]:
with open('/kaggle/input/btg-notas-de-corretagem-2022/BTG notas de corrertagem 2022.pdf', 'rb') as f:
    content = io.BytesIO(f.read())
    content.seek(0)
    
    brokerage_notes = B3Parser(brokerage_note=content).parse_brokerage_note()
    

In [55]:
notas_btg = pd.DataFrame(brokerage_notes)
notas_btg.head()

,reference_date,settlement_fee,registration_fee,term_fee,ana_fee,emoluments,operational_fee,execution,custody_fee,taxes,others,transactions
0,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am..."
1,2022-05-24,0.01,0.04,0.00,0.00,0.02,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am..."
2,2022-05-25,0.00,0.02,0.00,0.00,0.01,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am..."
3,2022-05-27,0.13,0.33,0.00,0.00,0.17,0,0.00,0,0,0,"[{'transaction_type': TransactionType.SELL, 'a..."
4,2022-05-30,0.04,0.11,0.00,0.00,0.06,0,0.00,0,0,0,"[{'transaction_type': TransactionType.SELL, 'a..."


In [56]:
with open('/kaggle/input/btg-notas-de-corretagem-2022/rama nota-de-corretagem-2022-combinado.pdf', 'rb') as f:
    content = io.BytesIO(f.read())
    content.seek(0)
    
    brokerage_notes = B3Parser(brokerage_note=content).parse_brokerage_note()

notas_orama = pd.DataFrame(brokerage_notes)
notas_orama.head()

,reference_date,settlement_fee,registration_fee,term_fee,ana_fee,emoluments,operational_fee,execution,custody_fee,taxes,others,transactions
0,2022-01-04,0,0,0,0,0,0,0,0,0,0,"[{'transaction_type': TransactionType.SELL, 'a..."
1,2022-01-07,0,0,0,0,0,0,0,0,0,0,"[{'transaction_type': TransactionType.SELL, 'a..."
2,2022-01-11,0,0,0,0,0,0,0,0,0,0,"[{'transaction_type': TransactionType.SELL, 'a..."
3,2022-01-14,0,0,0,0,0,0,0,0,0,0,"[{'transaction_type': TransactionType.SELL, 'a..."
4,2022-01-25,0,0,0,0,0,0,0,0,0,0,"[{'transaction_type': TransactionType.SELL, 'a..."


## Combinação dos arquivos

In [57]:
notas = pd.concat([notas_orama,notas_btg])
notas.shape

(108, 12)

## Normalização da coluna transactions

Note-se que a coluna `transactions` é composta por um dicionário.

Nos próximos passos, iremos abrir os dicionários e incorporar as colunas derivadas ao DataFrame `notas`.

In [58]:
transactions_exploded_df = notas[['transactions']].explode('transactions')
len(transactions_exploded_df)

213

In [59]:
print(len(transactions_exploded_df))
transactions_exploded_df.head()

213


,transactions
0,"{'transaction_type': TransactionType.SELL, 'am..."
0,"{'transaction_type': TransactionType.SELL, 'am..."
1,"{'transaction_type': TransactionType.SELL, 'am..."
1,"{'transaction_type': TransactionType.SELL, 'am..."
1,"{'transaction_type': TransactionType.SELL, 'am..."


In [60]:
print(type(transactions_exploded_df.iloc[0,0]))
transactions_exploded_df.iloc[0,0]

<class 'dict'>


{'transaction_type': <TransactionType.SELL: 'sell'>,
 'amount': 200,
 'unit_price': Decimal('3.80'),
 'security': {'name': 'OPCAO DE VENDA 04/22 BOVAP950 ISHARES BOVA',
  'ticker': None},
 'source_withheld_taxes': Decimal('0.04')}

In [61]:
transactions_normalized_df = pd.json_normalize(transactions_exploded_df['transactions'])
transactions_normalized_df.index = transactions_exploded_df.index
transactions_normalized_df = transactions_normalized_df.rename(columns={'security.name': 'security_name', 'security.ticker': 'security_ticker'})
transactions_normalized_df.head(15)


,transaction_type,amount,unit_price,source_withheld_taxes,security_name,security_ticker
0,TransactionType.SELL,200,3.80,0.04,OPCAO DE VENDA 04/22 BOVAP950 ISHARES BOVA,None
0,TransactionType.SELL,100,0.05,0.00,OPCAO DE COMPRA 01/22 SAPRA20 UNT N2 SANEPAR,None
1,TransactionType.SELL,100,1.05,0.01,OPCAO DE VENDA 02/22 KLBNN256 UNT N2 KLABIN S/A,None
1,TransactionType.SELL,100,12.00,0.06,OPCAO DE VENDA 01/22 VALEM108 ON NM VALE,None
1,TransactionType.SELL,100,2.01,0.01,OPCAO DE VENDA 02/22 WEGEN309 ON NM WEG,EGEN3
1,TransactionType.SELL,100,1.01,0.01,OPCAO DE VENDA 02/22 BBDCN206 PN N1 BRADESCO,None
2,TransactionType.SELL,1000,0.05,0.00,OPCAO DE COMPRA 02/22 MGLUB899 ON NM MAGAZ LUIZA,None
3,TransactionType.SELL,100,0.17,0.00,OPCAO DE VENDA 01/22 GGBRM286 PN N1 GERDAU,None
3,TransactionType.SELL,100,1.15,0.01,OPCAO DE VENDA 02/22 VBBRN205 ON NM VIBRA,None
4,TransactionType.SELL,100,0.35,0.00,OPCAO DE COMPRA 03/22 SAPRC205 UNT N2 SANEPAR,None


In [62]:
notas = pd.merge(left = notas, right = transactions_normalized_df, how='right', left_index=True, right_index=True)
notas.reset_index(inplace=True, drop=True)
notas.pop('transactions')
notas.head()

,reference_date,settlement_fee,registration_fee,term_fee,ana_fee,emoluments,operational_fee,execution,custody_fee,taxes,others,transaction_type,amount,unit_price,source_withheld_taxes,security_name,security_ticker
0,2022-01-04,0,0,0,0,0,0,0,0,0,0,TransactionType.SELL,200,3.80,0.04,OPCAO DE VENDA 04/22 BOVAP950 ISHARES BOVA,None
1,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,TransactionType.SELL,200,3.80,0.04,OPCAO DE VENDA 04/22 BOVAP950 ISHARES BOVA,None
2,2022-01-04,0,0,0,0,0,0,0,0,0,0,TransactionType.SELL,100,0.05,0.00,OPCAO DE COMPRA 01/22 SAPRA20 UNT N2 SANEPAR,None
3,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,TransactionType.SELL,100,0.05,0.00,OPCAO DE COMPRA 01/22 SAPRA20 UNT N2 SANEPAR,None
4,2022-01-04,0,0,0,0,0,0,0,0,0,0,TransactionType.BUY,200,0.78,0,DE VENDA 07/22 BBASS363 ON,BASS36


Agora vamos renomear as colunas conforme a nota de corregatem

In [63]:
notas = notas.rename(columns={'reference_date': 'data_op', 
                      'settlement_fee': 'tx_liq', 
                      'registration_fee': 'tx_regist', 
                      'term_fee': 'tx_termo', 
                      'ana_fee': 'tx_ana', 
                      'emoluments': 'emolumentos',
                     'operational_fee': 'tx_oper',
                     'execution':'execucao',
                     'custody_fee': 'tx_custodia',
                     'taxes': 'impostos',
                     'others': 'outros',
                     'transaction_type': 'tipo_transac',
                     'amount': 'quant',
                     'unit_price':'preco_unitario',
                     'source_withheld_taxes': 'irrf_fonte',
                     'security_name':'acao'})
notas.head()

,data_op,tx_liq,tx_regist,tx_termo,tx_ana,emolumentos,tx_oper,execucao,tx_custodia,impostos,outros,tipo_transac,quant,preco_unitario,irrf_fonte,acao,security_ticker
0,2022-01-04,0,0,0,0,0,0,0,0,0,0,TransactionType.SELL,200,3.80,0.04,OPCAO DE VENDA 04/22 BOVAP950 ISHARES BOVA,None
1,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,TransactionType.SELL,200,3.80,0.04,OPCAO DE VENDA 04/22 BOVAP950 ISHARES BOVA,None
2,2022-01-04,0,0,0,0,0,0,0,0,0,0,TransactionType.SELL,100,0.05,0.00,OPCAO DE COMPRA 01/22 SAPRA20 UNT N2 SANEPAR,None
3,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,TransactionType.SELL,100,0.05,0.00,OPCAO DE COMPRA 01/22 SAPRA20 UNT N2 SANEPAR,None
4,2022-01-04,0,0,0,0,0,0,0,0,0,0,TransactionType.BUY,200,0.78,0,DE VENDA 07/22 BBASS363 ON,BASS36


In [64]:
notas['tipo_transac2'] = notas['tipo_transac'].apply(lambda x: 'compra' if ('TransactionType.BUY' == str(x)) else 'venda')
notas[['tipo_transac','tipo_transac2']].sample(n=10)

,tipo_transac,tipo_transac2
195,TransactionType.BUY,compra
171,TransactionType.SELL,venda
23,TransactionType.SELL,venda
16,TransactionType.SELL,venda
282,TransactionType.SELL,venda
252,TransactionType.BUY,compra
112,TransactionType.SELL,venda
178,TransactionType.SELL,venda
175,TransactionType.SELL,venda
243,TransactionType.SELL,venda


In [65]:
notas.pop('tipo_transac')

0      TransactionType.SELL
1      TransactionType.SELL
2      TransactionType.SELL
3      TransactionType.SELL
4       TransactionType.BUY
               ...         
320     TransactionType.BUY
321    TransactionType.SELL
322    TransactionType.SELL
323     TransactionType.BUY
324    TransactionType.SELL
Name: tipo_transac, Length: 325, dtype: object

In [66]:
notas.sample(n=20)

,data_op,tx_liq,tx_regist,tx_termo,tx_ana,emolumentos,tx_oper,execucao,tx_custodia,impostos,outros,quant,preco_unitario,irrf_fonte,acao,security_ticker,tipo_transac2
218,2022-05-11,0,0,0,0,0,0,0,0,0,0,200,0.34,0.00,OPCAO DE VENDA 05/22 ENATQ220 ON NM ENAUTA PART,None,venda
134,2022-03-21,0,0,0,0,0,0,0,0,0,0,200,0.10,0.00,OPCAO DE VENDA 04/22 ENBRP215 ON NM ENERGIAS BR,None,venda
195,2022-07-29,0.77,1.95,0.00,0.00,1.03,0,0.00,0,0,0,200,1.13,0,DE VENDA 09/22 PETRU313 PN,ETRU31,compra
205,2022-08-04,0.15,0.39,0.00,0.00,0.21,0,0.00,0,0,0,500,0.67,0,DE VENDA 09/22 EMBRU130 ON,None,compra
231,2022-08-18,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,100,2.12,0,DE VENDA 10/22 PETRV424 PN,ETRV4,compra
130,2022-03-18,0,0,0,0,0,0,0,0,0,0,300,0.05,0.00,OPCAO DE VENDA 04/22 RAIZP650 PN N2 RAIZEN,None,venda
291,2022-11-01,0.02,0.05,0.00,0.00,0.03,0,0.00,0,0,0,100,0.85,0.00,DE COMPRA 11/22 KLBNK215 UNT,None,venda
256,2022-09-19,0.04,0.12,0.00,0.00,0.06,0,0.00,0,0,0,100,1.78,0.01,DE COMPRA 12/22 SULAL260 UNT,None,venda
176,2022-04-28,0,0,0,0,0,0,0,0,0,0,100,0.50,0.00,OPCAO DE COMPRA 06/22 EMBRF160 ON NM EMBRAER,None,venda
280,2022-10-20,0.25,0.65,0.00,0.00,0.34,0,0.00,0,0,0,500,0.93,0,DE COMPRA 12/22 B3SAL145 ON,None,compra


Os dados vindos da Orama vieram com o nome 'OPCAO' antes dos dados

In [69]:
notas['acao2'] = notas['acao'].apply(lambda x: x.replace('OPCAO',''))
notas.sample(n=20)

,data_op,tx_liq,tx_regist,tx_termo,tx_ana,emolumentos,tx_oper,execucao,tx_custodia,impostos,outros,quant,preco_unitario,irrf_fonte,acao,security_ticker,tipo_transac2,acao2
102,2022-02-25,0,0,0,0,0,0,0,0,0,0,200,0.80,0.01,OPCAO DE VENDA 04/22 ENBRP215 ON NM ENERGIAS BR,None,venda,DE VENDA 04/22 ENBRP215 ON NM ENERGIAS BR
123,2022-07-05,0.19,0.48,0.00,0.00,0.25,0,0.00,0,0,0,300,0.65,0.01,DE VENDA 07/22 KLBNS206 UNT,None,venda,DE VENDA 07/22 KLBNS206 UNT
258,2022-09-23,0.00,0.02,0.00,0.00,0.01,0,0.00,0,0,0,600,0.05,0,DE COMPRA 10/22 KLBNJ215 UNT,None,compra,DE COMPRA 10/22 KLBNJ215 UNT
157,2022-07-18,0.00,0.00,0.00,0.00,0.00,0,0.00,0,0,0,100,0.05,0,DE COMPRA 08/22 SAPRH204 UNT,None,compra,DE COMPRA 08/22 SAPRH204 UNT
224,2022-08-10,0.17,0.44,0.00,0.00,0.23,0,0.00,0,0,0,200,1.05,0.01,DE VENDA 09/22 BOVAU101,None,venda,DE VENDA 09/22 BOVAU101
91,2022-06-23,0.02,0.06,0.00,0.00,0.03,0,0.00,0,0,0,100,0.50,0.00,OPCAO DE VENDA 04/22 SULAP260 UNT N2 SUL AMERICA,None,venda,DE VENDA 04/22 SULAP260 UNT N2 SUL AMERICA
269,2022-10-11,0.25,0.64,0.00,0.00,0.34,0,0.00,0,0,0,300,0.70,0.01,DE VENDA 10/22 BOVAV108,None,venda,DE VENDA 10/22 BOVAV108
88,2022-02-21,0,0,0,0,0,0,0,0,0,0,100,0.75,0.00,DE COMPRA 08/22 SULAH255 UNT,None,venda,DE COMPRA 08/22 SULAH255 UNT
145,2022-07-13,0.01,0.04,0.00,0.00,0.02,0,0.00,0,0,0,100,0.32,0.00,OPCAO DE COMPRA 05/22 SAPRE220 UNT N2 SANEPAR,None,venda,DE COMPRA 05/22 SAPRE220 UNT N2 SANEPAR
136,2022-03-21,0,0,0,0,0,0,0,0,0,0,700,0.43,0,DE COMPRA 08/22 KLBNH211 UNT,None,compra,DE COMPRA 08/22 KLBNH211 UNT


In [72]:

notas['ticker'] = notas['acao2'].apply(lambda x: str(x.split()[3]))

notas['tipo_opcao'] = notas['acao2'].apply(lambda x: 'put' if str(x.split()[1]) == 'VENDA' else ('call' if str(x.split()[1]) == 'COMPRA' else 'None'))
notas['vencimento_opcao'] = notas['acao2'].apply(lambda x: str(x.split()[2]))
notas['total_operacao'] = notas['quant'] * notas['preco_unitario']
notas[['acao2','ticker','vencimento_opcao','tipo_opcao','quant','preco_unitario','total_operacao']]

,acao2,ticker,vencimento_opcao,tipo_opcao,tipo_transac2,quant,preco_unitario,total_operacao
0,DE VENDA 04/22 BOVAP950 ISHARES BOVA,BOVAP950,04/22,put,venda,200,3.80,760.00
1,DE VENDA 04/22 BOVAP950 ISHARES BOVA,BOVAP950,04/22,put,venda,200,3.80,760.00
2,DE COMPRA 01/22 SAPRA20 UNT N2 SANEPAR,SAPRA20,01/22,call,venda,100,0.05,5.00
3,DE COMPRA 01/22 SAPRA20 UNT N2 SANEPAR,SAPRA20,01/22,call,venda,100,0.05,5.00
4,DE VENDA 07/22 BBASS363 ON,BBASS363,07/22,put,compra,200,0.78,156.00
...,...,...,...,...,...,...,...,...
320,DE COMPRA 02/23 PETRB32 PN,PETRB32,02/23,call,compra,100,1.32,132.00
321,DE VENDA 02/23 PETRN270 PN,PETRN270,02/23,put,venda,200,2.59,518.00
322,DE VENDA 01/23 VIIAM350 ON,VIIAM350,01/23,put,venda,400,1.50,600.00
323,DE VENDA 12/22 VIIAX350 ON,VIIAX350,12/22,put,compra,400,1.55,620.00


In [99]:
notas['saldo_operacao'] = notas.tipo_transac2
notas['saldo_operacao'] = notas['saldo_operacao'].apply(lambda x: 1 if str(x) == 'compra' else -1)
notas['saldo_operacao'] = notas['saldo_operacao'] * notas['total_operacao']
notas[['tipo_transac2','quant','preco_unitario','saldo_operacao']]


,tipo_transac2,quant,preco_unitario,saldo_operacao
0,venda,200,3.80,-760.00
1,venda,200,3.80,-760.00
2,venda,100,0.05,-5.00
3,venda,100,0.05,-5.00
4,compra,200,0.78,156.00
...,...,...,...,...
320,compra,100,1.32,132.00
321,venda,200,2.59,-518.00
322,venda,400,1.50,-600.00
323,compra,400,1.55,620.00


In [112]:
notas['saldo_operacao'].info

<bound method Series.info of 0      -760.00
1      -760.00
2        -5.00
3        -5.00
4       156.00
        ...   
320     132.00
321    -518.00
322    -600.00
323     620.00
324      -1.00
Name: saldo_operacao, Length: 325, dtype: object>

In [105]:
notas[['data_op','saldo_operacao']].groupby(['data_op']).sum()

,saldo_operacao
data_op,
2022-01-04,-682.10
2022-01-07,-1545.00
2022-01-11,-19.00
2022-01-14,-613.00
2022-01-25,-207.00
...,...
2022-11-28,255.00
2022-12-05,40.00
2022-12-07,12.00


In [107]:
sns.barplot(data = notas,
           x = 'data_op',
           y = 'saldo_operacao')

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

# Exercício de opções